# Ax Service API with RayTune on PyTorch CNN

Ax integrates easily with different scheduling frameworks and distributed training frameworks. In this example, Ax-driven optimization is executed in a distributed fashion using [RayTune](https://ray.readthedocs.io/en/latest/tune.html). 

RayTune is a scalable framework for hyperparameter tuning that provides many state-of-the-art hyperparameter tuning algorithms and seamlessly scales from laptop to distributed cluster with fault tolerance. RayTune leverages [Ray](https://ray.readthedocs.io/)'s Actor API to provide asynchronous parallel and distributed execution.

Ray 'Actors' are a simple and clean abstraction for replicating your Python classes across multiple workers and nodes. Each hyperparameter evaluation is asynchronously executed on a separate Ray actor and reports intermediate training progress back to RayTune. Upon reporting, RayTune then uses this information to performs actions such as early termination, re-prioritization, or checkpointing.

In [1]:
import logging
from ray import tune
from ray.tune import track
from ray.tune.suggest.ax import AxSearch
logger = logging.getLogger(tune.__name__)  
logger.setLevel(level=logging.CRITICAL)  # Reduce the number of Ray warnings that are not relevant here.

ModuleNotFoundError: No module named 'ray'

In [2]:
import torch
import numpy as np

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.utils.tutorials.cnn_utils import load_mnist, train, evaluate


init_notebook_plotting()

[INFO 06-28 16:41:16] ipy_plotting: Injecting Plotly library into cell. Do not overwrite or delete cell.


## 1. Initialize client
We specify `enforce_sequential_optimization` as False, because Ray runs many trials in parallel. With the sequential optimization enforcement, `AxClient` would expect the first few trials to be completed with data before generating more trials.

When high parallelism is not required, it is best to enforce sequential optimization, as it allows for achieving optimal results in fewer (but sequential) trials. In cases where parallelism is important, such as with distributed training using Ray, we choose to forego minimizing resource utilization and run more trials in parallel.

In [3]:
ax = AxClient(enforce_sequential_optimization=False)

## 2. Set up experiment
Here we set up the search space and specify the objective; refer to the Ax API tutorials for more detail.

In [4]:
ax.create_experiment(
    name="mnist_experiment",
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {"name": "momentum", "type": "range", "bounds": [0.0, 1.0]},
    ],
    objective_name="mean_accuracy",
)

[INFO 06-28 16:41:16] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy. Iterations after 5 will take longer to generate due to model-fitting.


## 3. Define how to evaluate trials
Since we use the Ax Service API here, we evaluate the parameterizations that Ax suggests, using RayTune. The evaluation function follows its usual pattern, taking in a parameterization and outputting an objective value. For detail on evaluation functions, see [Trial Evaluation](https://ax.dev/docs/runner.html). 

In [5]:
def train_evaluate(parameterization):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, valid_loader, test_loader = load_mnist(data_path='~/.data')
    net = train(train_loader=train_loader, parameters=parameterization, dtype=torch.float, device=device)
    track.log(
        mean_accuracy=evaluate(
            net=net,
            data_loader=valid_loader,
            dtype=torch.float,
            device=device,
        )
    )

## 4. Run optimization
Execute the Ax optimization and trial evaluation in RayTune using [AxSearch algorithm](https://ray.readthedocs.io/en/latest/tune-searchalg.html#ax-search):

In [6]:
tune.run(
    train_evaluate, 
    num_samples=30, 
    search_alg=AxSearch(ax),  # Note that the argument here is the `AxClient`.
    verbose=0,  # Set this level to 1 to see status updates and to 2 to also see trial results.
    # To use GPU, specify: resources_per_trial={"gpu": 1}.
)

2019-06-28 16:41:16,284	WARNING worker.py:1331 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-06-28 16:41:16,286	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-28_16-41-16_286272_48449/logs.
2019-06-28 16:41:16,408	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:13143 to respond...
2019-06-28 16:41:16,544	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:15265 to respond...
2019-06-28 16:41:16,548	INFO services.py:806 -- Starting Redis shard with 3.44 GB max memory.
2019-06-28 16:41:16,612	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-28_16-41-16_286272_48449/logs.
2019-06-28 16:41:16,616	INFO services.py:1442 -- Starting the Plasma object store with 5.15 GB memory using /tmp.


[train_evaluate_1_lr=0.003839,momentum=0.13058,
 train_evaluate_2_lr=0.040651,momentum=0.93145,
 train_evaluate_3_lr=0.00029751,momentum=0.42034,
 train_evaluate_4_lr=5.0684e-05,momentum=0.87119,
 train_evaluate_5_lr=0.16561,momentum=0.3552,
 train_evaluate_6_lr=0.0030413,momentum=0.58571,
 train_evaluate_7_lr=2.1341e-05,momentum=0.062879,
 train_evaluate_8_lr=6.4852e-06,momentum=0.9718,
 train_evaluate_9_lr=0.00087986,momentum=0.48901,
 train_evaluate_10_lr=0.2652,momentum=0.69606,
 train_evaluate_11_lr=6.9975e-05,momentum=0.21816,
 train_evaluate_12_lr=0.00019538,momentum=0.51276,
 train_evaluate_13_lr=0.00018449,momentum=1.8421e-17,
 train_evaluate_14_lr=0.0010596,momentum=0.0,
 train_evaluate_15_lr=0.0035145,momentum=7.3464e-16,
 train_evaluate_16_lr=0.00030799,momentum=0.22074,
 train_evaluate_17_lr=0.00038031,momentum=0.18887,
 train_evaluate_18_lr=1.5967e-05,momentum=0.56366,
 train_evaluate_19_lr=0.00064806,momentum=0.080248,
 train_evaluate_20_lr=0.0010568,momentum=1.0195e-16,

## 5. Retrieve the optimization results

In [7]:
best_parameters, values = ax.get_best_parameters()
best_parameters

{'lr': 0.0035144522261298635, 'momentum': 7.346423042648887e-16}

In [8]:
means, covariances = values
means

{'mean_accuracy': 0.969666685940424}

## 6. Plot the response surface and optimization trace

In [9]:
render(
    plot_contour(
        model=ax.generation_strategy.model, param_x='lr', param_y='momentum', metric_name='mean_accuracy'
    )
)

In [10]:
# `plot_single_method` expects a 2-d array of means, because it expects to average means from multiple 
# optimization runs, so we wrap out best objectives array in another array.
best_objectives = np.array([[trial.objective_mean * 100 for trial in ax.experiment.trials.values()]])
best_objective_plot = optimization_trace_single_method(
    y=np.maximum.accumulate(best_objectives, axis=1),
    title="Model performance vs. # of iterations",
    ylabel="Accuracy",
)
render(best_objective_plot)